# Install Packages

In [2]:
! pip install -q python-dotenv pydantic 
! pip install -q agno

# Load .env File and Access Keys

In [4]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel, SecretStr

# Load .env file
load_dotenv()

# Define a Pydantic model to structure the API keys
class APIKeys(BaseModel):
    openai_api_key: SecretStr
    groq_api_key: SecretStr
    tavily_api_key: SecretStr

# Fetch keys from environment
keys = APIKeys(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    groq_api_key=os.getenv("GROQ_API_KEY"),        # Corrected: GROQ instead of Groq
    tavily_api_key=os.getenv("TAVILY_API_KEY"),
)

# Check presence of each secret
openai_key_present = bool(keys.openai_api_key.get_secret_value())
groq_key_present = bool(keys.groq_api_key.get_secret_value())
tavily_key_present = bool(keys.tavily_api_key.get_secret_value())

# Show results as booleans
print("OpenAI Key Present:", openai_key_present)
print("Groq Key Present:", groq_key_present)
print("Tavily Key Present:", tavily_key_present)


OpenAI Key Present: True
Groq Key Present: True
Tavily Key Present: True


# Initiate Agent

## Short-Term Memory

- `SqliteStorage`	Persists session memory to disk (so memory survives restarts)
- `session_id`	Links all interactions together under one conversational session
- `add_history_to_messages=True`	Ensures agent sees previous exchanges (context)
- `agent.run()`	Sends user input to the LLM with relevant context
- `response.content`	Retrieves and displays the model's reply

In [7]:
# Create a Memory-Aware Agent

# 1. Import necessary classes from Agno
from agno.agent import Agent                            # Core agent framework
from agno.models.openai import OpenAIChat               # OpenAI model wrapper
from agno.storage.sqlite import SqliteStorage           # Persistent memory via SQLite

# 2. Instantiate the agent with memory support
agent = Agent(
    model=OpenAIChat(id="gpt-4o"),                      # Set the model (GPT-4o or GPT-3.5)
    
    # Use SQLite to persist memory (conversation history)
    storage=SqliteStorage(
        table_name="agent_sessions",                    # Table to store sessions
        db_file="agent_memory.db"                       # SQLite database file
    ),
    
    session_id="test_session_1",                        # Identifies this user/session uniquely
    
    add_history_to_messages=True,                       # Include chat history in the prompt
    add_datetime_to_instructions=True                   # Add current datetime to help reasoning
)


###💬 Step-by-Step: Interact with the Agent

In [9]:
# 3. Run a simple input-response loop
while True:
    user_input = input("You: ")                         # Get user input from console
    
    if user_input.lower() in ["exit", "quit"]:          # Graceful exit condition
        break

    response = agent.run(user_input)                    # Send user input to agent and get reply
    print("Agent:", response.content)                   # Print model's response


You:  Hello, My name is Nayeem and I am from Dhaka. Can you tell me about Paris?


Agent: Hello Nayeem! I'd be happy to tell you about Paris. Paris, the capital city of France, is renowned for its iconic landmarks, world-class museums, and rich cultural heritage. Here are some highlights:

1. **Landmarks**: Paris is home to many famous landmarks, including the Eiffel Tower, the Notre-Dame Cathedral, the Arc de Triomphe, and the Sacré-Cœur Basilica. The Eiffel Tower, built in 1889, is one of the most recognizable structures in the world.

2. **Museums**: The city boasts a wealth of museums such as the Louvre, which houses a vast collection of art and artifacts, including the Mona Lisa and the Venus de Milo. The Musée d'Orsay is famous for its Impressionist works by artists like Monet and Van Gogh.

3. **Culture**: Known as the "City of Light," Paris has a vibrant cultural scene with a rich history in art, fashion, and cuisine. The city is famous for its café culture, and you can often see Parisians enjoying coffee at charming sidewalk cafés.

4. **Fashion**: Paris is 

You:  how much is 2+2?


Agent: 2 + 2 equals 4.


You:  tell me what my name means?


Agent: The name "Nayeem" has Arabic origins and is commonly used in various cultures, particularly in Muslim communities. It often means "comfort, ease, or tranquility." As with many names, the exact meaning and significance can vary somewhat depending on cultural and regional differences. It's a name that carries a positive connotation, suggesting peace and well-being.


You:  exit


## Test Memory Persistence Across Restarts

In [17]:
response = agent.run("What city do I live in?")
print("Agent:", response.content)

Agent: You mentioned earlier that you are from Dhaka, so I assume you live there. Dhaka is the capital of Bangladesh and is known for its vibrant culture, history, and as a major center for commerce and education in the country. If you are referring to a different city or have any more specific questions about Dhaka or another topic, feel free to let me know!
